# Exploring new features of OpenAI's GPT-5 model
In this tutorial, we’ll explore the new capabilities introduced in OpenAI’s latest model, GPT-5. The update brings several powerful features, including the Verbosity parameter, Free-form Function Calling, Context-Free Grammar (CFG), and Minimal Reasoning. We’ll look at what they do and how to use them in practice.

## Installing the libraries

In [1]:
!pip install pandas openai

To get an OpenAI API key, visit https://platform.openai.com/settings/organization/api-keys and generate a new key. If you're a new user, you may need to add billing details and make a minimum payment of $5 to activate API access.

In [2]:
import os
from getpass import getpass
os.environ['OPENAI_API_KEY'] = getpass('Enter OpenAI API Key: ')

Enter OpenAI API Key: ··········


## 1. Verbosity Parameter
The Verbosity parameter lets you control how detailed the model’s replies are without changing your prompt.

* low → Short and concise, minimal extra text.

* medium (default) → Balanced detail and clarity.

* high → Very detailed, ideal for explanations, audits, or teaching.

In [ ]:
from openai import OpenAI
import pandas as pd
from IPython.display import display

client = OpenAI()

question = "Write a poem about a detective and his first solve"

data = []

for verbosity in ["low", "medium", "high"]:
    response = client.responses.create(
        model="gpt-5-mini",
        input=question,
        text={"verbosity": verbosity}
    )

    # Extract text
    output_text = ""
    for item in response.output:
        if hasattr(item, "content"):
            for content in item.content:
                if hasattr(content, "text"):
                    output_text += content.text

    usage = response.usage
    data.append({
        "Verbosity": verbosity,
        "Sample Output": output_text,
        "Output Tokens": usage.output_tokens
    })


In [4]:
# Create DataFrame
df = pd.DataFrame(data)

# Display nicely with centered headers
pd.set_option('display.max_colwidth', None)
styled_df = df.style.set_table_styles(
    [
        {'selector': 'th', 'props': [('text-align', 'center')]},  # Center column headers
        {'selector': 'td', 'props': [('text-align', 'left')]}     # Left-align table cells
    ]
)

display(styled_df)

,Verbosity,Sample Output,Output Tokens
0,low,"He learned the city by its silences — rain in gutters, neon sighing, a paper cup spinning in a puddle like a clock hand. His coat still smelled of factory floors and new breath; the badge felt heavier than the coin his father gave him. The case was crooked in the way people hide small things: a borrowed alibi, a burned receipt, a photograph folded in a pocket. He sat with evidence like a patient animal, waiting. Clues arrived as soft as footsteps — a stamp, a thread of tobacco. A witness stammered a truth between apologies; a ledger opened like a palm, numbers falling into place. He fit the fragments together: motive, time, an overheard lie. It clicked — quiet as a door unlocked — and the room exhaled. The first solve did not make him a hero, only a clearer-eyed man. He learned how guilt looks when stripped of theater, how relief can be a small, weary thing. He went home with mud on his shoes and a new respect for the dark.",731
1,medium,"The badge was new and cold against my palm, a metal question I had not yet learned to ask. Night draped the block in the same tired coat the city wore to pretend nothing had happened. They sent me to a room where words had been used up— a window steamed with someone else's breath, a child's shoe tucked beneath the radiator, a smear of something that did not belong. I knelt and felt for patterns as if for pulse. Clues are small kindnesses from the world, or cruelties; the first one looked like an accident until it didn't. I lined facts up like dominoes and watched the truth give way beneath the weight of evidence. There was a man who said the right thing twice, a woman who blinked when she thought no one watched, a ledger scribbled in a hand that trembled. Solving it was not a sudden light but a sequence: the slow importuning of questions, the soft unspooling of alibis, the way lies fold to fit themselves like paper in a drawer. The confession came out of a cigarette and a long dry silence, not with thunder but with the ordinary sound of someone unloading what they could not carry. When they pinned the name on the board, my hands stopped shaking and started remembering— how the street smelled then, how rain makes the neon honest, how a small shoe sits forever at the lip of a radiator. The first solve gave me a key and a wound: I learned how to pry at seams and watch people show themselves as if by accident. I learned the cost. Since that night, each case begins the same way— with a cold badge and a reluctant city—and ends with a quiet window opened to let truth out, or a door shut on the things we cannot make clean.",1017
2,high,"He learned to listen the way some men learn to pray: not with words, but with the elbows of his palms, laying them on cold surfaces of kitchen tables and car hoods, feeling the small, honest tremors of things left behind. The case came in on a slow, rainy Tuesday, a missing watch, brass and grief in equal measure— an old man's heirloom, wound tight with decades, gone from a dresser like a punctuation mark erased. They joked about it at the station, chewed stale coffee, but he kept the file like an unfinished sentence. He read the witness statements as if they were maps, tracing the same crooked lines until the ink faded true. A pattern sat patient in the margins: the neighbor’s cat, a lemon peel on the sink, a thread of blue lint tangled in the hem of a coat. He learned that clues are shy things, shy like children, that you invite out by the tone of your question, the steadiness of your gaze. So he went back to the house at dusk, smelling the rain and the lemon peel, running his thumb along drawer edges, reading dust like a palm reader, noting where the light had been allowed to fall. There in the seam of an old atlas book—how ridiculous—he found the watch, nestled like a secret between pages of places no one promised to visit. It was cooler than he expected, stamped by time, wound low but not dead. A bus ticket fluttered 

The output tokens scale roughly linearly with verbosity: low (731) → medium (1017) → high (1263).

## 2. Free-Form Function Calling
Free-form function calling lets GPT-5 send raw text payloads—like Python scripts, SQL queries, or shell commands—directly to your tool, without the JSON formatting used in GPT-4.

This makes it easier to connect GPT-5 to external runtimes such as:

* Code sandboxes (Python, C++, Java, etc.)

* SQL databases (outputs raw SQL directly)

* Shell environments (outputs ready-to-run Bash)

* Config generators

In [11]:
from openai import OpenAI

client = OpenAI()

response = client.responses.create(
    model="gpt-5-mini",
    input="Please use the code_exec tool to calculate the cube of the number of vowels in the word 'pineapple'",
    text={"format": {"type": "text"}},
    tools=[
        {
            "type": "custom",
            "name": "code_exec",
            "description": "Executes arbitrary python code",
        }
    ]
)

[ResponseReasoningItem(id='rs_6896258896d4819581704bc526cbe75f01010511b1d90111', summary=[], type='reasoning', encrypted_content=None, status=None), ResponseOutputMessage(id='ctc_6896258aeb4c8195acbd14063b08687701010511b1d90111', content=None, role=None, status='completed', type='custom_tool_call', call_id='call_OGv2f6N10xi3dF8MW4AsLE5C', input='word = "pineapple"\nvowels = set("aeiouAEIOU")\ncount = sum(1 for ch in word if ch in vowels)\ncube = count ** 3\nprint(count)\nprint(cube)\n', name='code_exec')]


In [15]:
print(response.output[1].input)

word = "pineapple"
vowels = set("aeiouAEIOU")
count = sum(1 for ch in word if ch in vowels)
cube = count ** 3
print(count)
print(cube)



This output shows GPT-5 generating raw Python code that counts the vowels in the word pineapple, calculates the cube of that count, and prints both values. Instead of returning a structured JSON object (like GPT-4 typically would for tool calls), GPT-5 delivers plain executable code. This makes it possible to feed the result directly into a Python runtime without extra parsing.

## 3. Context-Free Grammar (CFG)
A Context-Free Grammar (CFG) is a set of production rules that define valid strings in a language. Each rule rewrites a non-terminal symbol into terminals and/or other non-terminals, without depending on the surrounding context.

CFGs are useful when you want to strictly constrain the model’s output so it always follows the syntax of a programming language, data format, or other structured text — for example, ensuring generated SQL, JSON, or code is always syntactically correct.

For comparison, we’ll run the same script using GPT-4 and GPT-5 with an identical CFG to see how both models adhere to the grammar rules and how their outputs differ in accuracy and speed.

In [17]:
from openai import OpenAI
import re

client = OpenAI()

email_regex = r"^[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}$"

prompt = "Give me a valid email address for John Doe. It can be a dummy email"

# No grammar constraints — model might give prose or invalid format
response = client.responses.create(
    model="gpt-4o",  # or earlier
    input=prompt
)

output = response.output_text.strip()
print("GPT Output:", output)
print("Valid?", bool(re.match(email_regex, output)))

GPT Output: Sure, here's a test email you can use for John Doe: johndoe@example.com
Valid? False


In [18]:
from openai import OpenAI

client = OpenAI()

email_regex = r"^[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}$"

prompt = "Give me a valid email address for John Doe. It can be a dummy email"

response = client.responses.create(
    model="gpt-5",  # grammar-constrained model
    input=prompt,
    text={"format": {"type": "text"}},
    tools=[
        {
            "type": "custom",
            "name": "email_grammar",
            "description": "Outputs a valid email address.",
            "format": {
                "type": "grammar",
                "syntax": "regex",
                "definition": email_regex
            }
        }
    ],
    parallel_tool_calls=False
)

print("GPT-5 Output:", response.output[1].input)


GPT-5 Output: john.doe@example.com


This example shows how GPT-5 can adhere more closely to a specified format when using a Context-Free Grammar.

With the same grammar rules, GPT-4 produced extra text around the email address ("Sure, here's a test email you can use for John Doe: johndoe@example.com"), which makes it invalid according to the strict format requirement.

GPT-5, however, output exactly john.doe@example.com, matching the grammar and passing validation. This demonstrates GPT-5’s improved ability to follow CFG constraints precisely.

## 4. Minimal Reasoning
Minimal reasoning mode runs GPT-5 with very few or no reasoning tokens, reducing latency and delivering a faster time-to-first-token.

It’s ideal for deterministic, lightweight tasks such as:

* Data extraction

* Formatting

* Short rewrites

* Simple classification

Because the model skips most intermediate reasoning steps, responses are quick and concise. If not specified, the reasoning effort defaults to medium.

In [21]:
import time
from openai import OpenAI

client = OpenAI()

prompt = "Classify the given number as odd or even. Return one word only."

start_time = time.time()  # Start timer

response = client.responses.create(
    model="gpt-5",
    input=[
        { "role": "developer", "content": prompt },
        { "role": "user", "content": "57" }
    ],
    reasoning={
        "effort": "minimal"  # Faster time-to-first-token
    },
)

latency = time.time() - start_time  # End timer

# Extract model's text output
output_text = ""
for item in response.output:
    if hasattr(item, "content"):
        for content in item.content:
            if hasattr(content, "text"):
                output_text += content.text

print("--------------------------------")
print("Output:", output_text)
print(f"Latency: {latency:.3f} seconds")


--------------------------------
Output: odd
Latency: 1.021 seconds
